In [1]:
import pandas as pd
import json
from datetime import datetime
import datetime
import math
from pandas.io.json import json_normalize #package for flattening json in pandas df


In [2]:
csv = pd.read_csv("data/companies_stock_price_worldwide.csv")

In [3]:
dates = list(csv['Date'].unique())
dates = pd.Series(dates)

Preprocess the dataset

In [4]:
def preprocess(csv):
    csv = csv.drop_duplicates(subset = ['Date','Company'], keep = 'first')
    list_companies = list(csv['Company'].unique())

    l = []
    for c in list_companies:
        if len(csv.loc[csv['Company'] == c]) < 300:
            csv = csv[csv['Company'] != c]
    
    csv = csv.loc[csv['Date'] != '2020-12-25']
    csv = csv.loc[csv['Date'] != '2020-09-07']
    csv = csv.loc[csv['Date'] != '2020-09-18']
    csv = csv.loc[csv['Date'] != '2020-04-10']
    csv = csv.loc[csv['Date'] != '2019-09-02']
    csv = csv.loc[csv['Date'] != '2019-12-25']
    csv = csv.loc[csv['Date'] != '2020-02-17']
    csv = csv.loc[csv['Date'] != '2020-07-03']
    csv = csv.loc[csv['Date'] != '2020-01-20']
    csv = csv.loc[csv['Date'] != '2019-11-28']
    csv = csv.loc[csv['Date'] != '2020-11-28']
    csv = csv.loc[csv['Date'] != '2020-05-01']
    csv = csv.loc[csv['Date'] != '2020-04-13']
    csv = csv.loc[csv['Date'] != '2019-12-26']
    csv = csv.loc[csv['Date'] != '2020-12-26']
    csv = csv.loc[csv['Date'] != '2020-05-25']
    csv = csv.loc[csv['Date'] != '2019-12-24']
    csv = csv.loc[csv['Date'] != '2019-12-31']
    csv = csv.loc[csv['Date'] != '2020-12-24']
    csv = csv.loc[csv['Date'] != '2020-12-31']
    csv = csv.loc[csv['Date'] != '2020-07-20']
    return csv

Add dates array to a dictionary

In [5]:
def add_dates_to_dict(j,csv):
    dates = list(csv['Date'].unique())

    # Sort the dates
    dates = pd.Series(dates)
    dates = pd.to_datetime(dates)
    dates = dates.sort_values()

    # Create the object array
    obj_dates = {}
    for i in range(len(dates)):
        obj_dates[i] = str(dates[i]).split(' ')[0]

    # Add it to the json structure    
    j['dates'] = obj_dates
    return j

Adds sector array to a dictionary

In [6]:
def add_sectors_to_dict(j,csv):
    list_sectors = list(csv['Sector'].unique())
    sec_list = []
    for sect in list_sectors:
        sec = {'name':sect, 'companies': []}
        sec['companies'] = add_companies_to_dict(j,csv.loc[csv['Sector'] == sect])
        print(str(len(sec['companies'])) + 'companies add in ' + sect)
        sec_list.append(sec)
    j['sectors'] = sec_list
    return j

Add a company stock to an array of companies

In [7]:
def add_company_to_dict(j,csv,c):
    whole_dates = pd.DataFrame()
    present_dates = pd.DataFrame()
    # Takes the the date set
    whole_dates['date'] = j['dates'].values()
    present_dates['date'] = csv['Date']
    missed_dates = list(whole_dates.loc[~whole_dates['date'].isin(present_dates['date'])]['date'])
    #add missing stock dates
    new_rows = pd.DataFrame(columns = csv.columns)
    new_rows['Date'] = missed_dates
    new_rows['Open'] = float('Inf')
    csv = csv.append(new_rows)
    csv['Date'] = pd.to_datetime(csv['Date'])
    csv = csv.sort_values(by = 'Date')
    csv = csv.reset_index(drop = True)
    for i,row in csv.iterrows():
        if math.isinf(row['Open']):
            csv.iloc[i,1] = csv.iloc[i-1,1]
            csv.iloc[i,2] = csv.iloc[i-1,2]
            csv.iloc[i,3] = csv.iloc[i-1,3]
            csv.iloc[i,4] = csv.iloc[i-1,4]
            csv.iloc[i,5] = csv.iloc[i-1,5]
            csv.iloc[i,6] = csv.iloc[i-1,6]
            csv.iloc[i,7] = csv.iloc[i-1,7]
            csv.iloc[i,8] = csv.iloc[i-1,8]
    # Create the company object
    company_obj = {}
    company_obj['name'] = c
    company_obj['stock'] = list(csv['Close'])
    return company_obj

Add companies to each sector dictionary

In [8]:
def add_companies_to_dict(j,csv):
    list_companies = list(csv['Company'].unique())
    companies_list = []
    for c in list_companies:
        csv_company = csv.loc[csv['Company'] == c]
        companies_list.append(add_company_to_dict(j,csv_company,c))
    return companies_list

Create the dictionary

In [9]:
def dict_creator(csv):
    j = {
        'name' : 'stock market', 
        'sectors':[], 
        'dates': []} 
    j = add_dates_to_dict(j,csv)
    j = add_sectors_to_dict(j,csv)
    return j

Main process

In [10]:
# Prepare and create the json
prep_csv = preprocess(csv)
js = dict_creator(prep_csv)
#json_object = json.dumps(js, indent = 4) 

# Writing to sample.json 
#with open("sample.json", "w") as outfile: 
#    outfile.write(json_object) 


88companies add in Financial Services
80companies add in Technology
29companies add in Communication Services
33companies add in Utilities
30companies add in Energy
77companies add in Consumer Cyclical
28companies add in Basic Materials
41companies add in Consumer Defensive
68companies add in Healthcare
78companies add in Industrials
35companies add in Real Estate


In [11]:
# Back to df and save as csv
df = pd.DataFrame()
sectors = js['sectors']
for el in sectors:
    for comp in el['companies']:
        temp = pd.DataFrame()
        temp['date'] = list(js['dates'].values())
        temp['stock'] = comp['stock']
        temp['company'] = comp['name']
        df = df.append(temp)
df = df.reset_index(drop = True)
df.to_csv('./data/companies/comp.csv',index = False, sep = '\t')